<a href="https://colab.research.google.com/github/thecoder-001/MineColab/blob/ngrok-mitigation/MineColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
███╗   ███╗██╗███╗   ██╗███████╗ ██████╗ ██████╗ ██╗      █████╗ ██████╗ 
████╗ ████║██║████╗  ██║██╔════╝██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
██╔████╔██║██║██╔██╗ ██║█████╗  ██║     ██║   ██║██║     ███████║██████╔╝
██║╚██╔╝██║██║██║╚██╗██║██╔══╝  ██║     ██║   ██║██║     ██╔══██║██╔══██╗
██║ ╚═╝ ██║██║██║ ╚████║███████╗╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
╚═╝     ╚═╝╚═╝╚═╝  ╚═══╝╚══════╝ ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝ 
```
**Run a Minecraft Server on Google Colab!**

---

The script below will run your server. You'll have to create a server first to be able to use it - don't worry, the scripts below will do the majority of the work for you. You might also want to change the default region to your region, check below.

In [ ]:
!pip -q install pyngrok

import os
import re
import getpass
import subprocess
from pyngrok import conf, ngrok
# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"
# Install OpenJDK 15
!wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
!sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk15 gets installed."
!sudo apt-get install adoptopenjdk-15-hotspot-jre &>/dev/null && echo "Yay! Openjdk15 has been successfully installed." || echo "Failed to install OpenJdk15."
#Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "15" :
  print("Openjdk15 is working correctly, you are good to go.")
else:
  print("Openjdk15 doesn't seems to be installed or isn't working, falling back to java", java_ver[0], ". You might experience reduced performance.")


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

#set the tunnel service
tunnel_service = "lynk" #available services: ngrok, lynk, tunnelto

# Ask for the ngrok authtoken
if tunnel_service == "ngrok" :
  print("Get your authtoken from https://dashboard.ngrok.com/auth")
  authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  !ngrok authtoken $authtoken # login to ngrok
elif tunnel_service == "lynk" :
  !wget https://dl.loopholelabs.io/releases/lynk/linux/0.0.1-beta/x64/lynk.zip
  print("get yout token and key from https://lynk.sh/dashboard")
  lynk_token = getpass.getpass()  # input your lynk token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  lynk_key = getpass.getpass()
  !chmod +x lynk
  !./lynk auth -t $lynk_token -k $lynk_key
  with open("/content/lynk_temp.txt") as out :
    subprocess.Popen(["./lynk", "tcp", "25566"], stdout=out)
  print("test")
elif tunnel_service == "tunnelto" :
  print("todo") #todo



# Sets default ngrok region
conf.get_default().region = 'us'  # Change this to whichever region you want

# Connect to ngrok
url = ngrok.connect(25565, 'tcp')
print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/My Drive/Minecraft-server"
!ls #list the directory contents (to verify that working directory was changed)

print('Starting server...')
!java -Xmx6144M -Xms6144M -XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true -jar server.jar nogui


# Make-a-Server

The code below will download a server for you and accept the EULA. After running these scripts, your server will be ready to run.

**Download the Minecraft server**

The code below will download Paper, a high-performance fork of the Vanilla server.

In [ ]:
# Replace "1.16.4" with your desired server version.
# Available versions:
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8
# Newer versions might work too, however this isn't guaranteed.
version = '1.16.4'

from google.colab import drive
import requests

drive.mount('/content/drive')

# Create the directory which will be used for the server
! mkdir "/content/drive/My Drive/Minecraft-server"

print('Downloading to Google Drive...')

url = 'https://papermc.io/api/v1/paper/' + version + '/latest/download'

r = requests.get(url)

if r.status_code is 200:
  with open('/content/drive/My Drive/Minecraft-server/server.jar', 'wb') as f:
    f.write(r.content)
  print('Done!')
else:
  print('Error '+ str(r.status_code) + '! Most likely you entered a unsupported version. Try running the code again if you think that shouldn\'t have happened.')

**Automatically accept the EULA**

In [ ]:
# Please read the file stored in your server folder before running this command. 
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-server"
! sed -i 's/eula=false/eula=true/g' eula.txt

# Debug
SSH acess to host OS

In [ ]:
## For SSH acess to host OS ##
#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:minecraft123 | chpasswd #change the password (default is minecraft123)
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass() #input your Ngrok auth token everytime or simply replace "getpass.getpass()" with your token in "double quotes"

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

#Get the public address by running the cell below (for some reason, running both the commands in same cell result in an error. 
#Thus it has been kept in a sepreate cell)

In [ ]:
#Get public address 
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
## For inspecting the minecraft server directory ##
%cd "/content/drive/My Drive/Minecraft-server"
!ls


#test

In [2]:
#Testing
!pip -q install pyngrok

import os
import re
import getpass
import subprocess
import time

from pyngrok import conf, ngrok

tunnel_service = "lynk" #available services: ngrok, lynk, tunnelto

# Ask for the ngrok authtoken
if tunnel_service == "ngrok" :
  print("Get your authtoken from https://dashboard.ngrok.com/auth")
  import getpass
  authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  !ngrok authtoken $authtoken # login to ngrok
elif tunnel_service == "lynk" :
  !wget https://dl.loopholelabs.io/releases/lynk/linux/0.0.1-beta/x64/lynk.zip && unzip lynk.zip
  print("get yout token and key from https://lynk.sh/dashboard")
  lynk_token = getpass.getpass()  # input your lynk token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  lynk_key = getpass.getpass()
  !chmod +x lynk
  !./lynk auth -t $lynk_token -k $lynk_key
  !touch lynk_temp_old.txt ; mv lynk_temp.txt lynk_temp_old.txt ; touch lynk_temp.txt
  time.sleep(2.4)
  with open("/content/lynk_temp.txt") as out :
    subprocess.Popen(["./lynk", "tcp", "25566"], stdout=out)
  print("test")
  with open("/content/lynk_temp.txt") as new , open("/content/lynk_temp_old.txt") as old :
    if new.read() == old.read() :
      print("Oops! something went wrong. You would need to run the script again")
      exit()
    else :
      print(new.read())



--2021-02-04 07:52:05--  https://dl.loopholelabs.io/releases/lynk/linux/0.0.1-beta/x64/lynk.zip
Resolving dl.loopholelabs.io (dl.loopholelabs.io)... 205.185.216.42, 205.185.216.10
Connecting to dl.loopholelabs.io (dl.loopholelabs.io)|205.185.216.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19703770 (19M) [application/zip]
Saving to: ‘lynk.zip.1’

lynk.zip.1          100%[===================>]  18.79M  --.-KB/s    in 0.09s   

2021-02-04 07:52:05 (199 MB/s) - ‘lynk.zip.1’ saved [19703770/19703770]

Archive:  lynk.zip
replace lynk? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: lynk                    
get yout token and key from https://lynk.sh/dashboard
··········
··········
██╗     ██╗   ██╗███╗   ██╗██╗  ██╗
██║     ╚██╗ ██╔╝████╗  ██║██║ ██╔╝
██║      ╚████╔╝ ██╔██╗ ██║█████╔╝ 
██║       ╚██╔╝  ██║╚██╗██║██╔═██╗ 
███████╗   ██║   ██║ ╚████║██║  ██╗
╚══════╝   ╚═╝   ╚═╝  ╚═══╝╚═╝  ╚═╝
                                   
Loophole Labs Inc.
✔  succ

FileNotFoundError: ignored

In [ ]:
!pip -q install pyngrok

import os
import re
import getpass
import subprocess
from pyngrok import conf, ngrok
!./lynk tcp 25566 > lynk_temp.txt

⠋ Preparing Lynk Client
⠋ Connecting to Lynk Server at lb.lynk.sh
⠋ Retreiving Lynk Server
⠋ Authenticating...
⠙ Authenticating...
⠹ Authenticating...
^C
